In [2]:
!pip install scikit-learn

     ---------------------------------------- 7.4/7.4 MB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 38.6/38.6 MB 5.5 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 4.6 MB/s eta 0:00:00


In [6]:
import torch
from torch.optim import Adam
from torch.nn import Linear, MSELoss, Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston

In [12]:
boston = load_boston()
boston

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [13]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [11]:
print(boston['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [15]:
df = pd.DataFrame(boston['data'])
df.columns = boston['feature_names']
df['medv'] = boston['target']
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,medv
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


### Practice
- 조건: 하이레벨로   
- 특성데이터 0번째 행에 대한 medv 예측값을 구하시요.

In [55]:
#df.iloc[행, 열]
x_temp = df.iloc[:,:-1].values 
x_len = len(x_temp[0])
print('x_len', x_len)
x = torch.FloatTensor(x_temp) # x는 특성 데이터
y = torch.FloatTensor(df.iloc[:,[-1]].values) # y는 레이블

x_len 13


In [56]:
x.shape

torch.Size([506, 13])

In [58]:
model = Sequential()
model.add_module('nn1', Linear(x_len, 1))
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=0.1)

In [59]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1441, -0.2364,  0.0482,  0.2306,  0.1799, -0.0591, -0.2512,  0.1051,
          -0.0625,  0.1785,  0.0584,  0.0970,  0.2105]], requires_grad=True),
 Parameter containing:
 tensor([0.1701], requires_grad=True)]

In [60]:
for step in range(1000):
    optimizer.zero_grad()
    hx = model.forward(x) # matmul (x, w) + b # model(x) 가능
    cost = loss_fn(hx, y)
    cost.backward()
    optimizer.step()
    print(step, cost.item())

0 5886.68896484375
1 813.5936279296875
2 4156.7392578125
3 2809.7880859375
4 552.803466796875
5 531.28125
6 1788.52587890625
7 1983.04150390625
8 1014.3043212890625
9 203.76502990722656
10 349.05224609375
11 990.5137329101562
12 1196.5264892578125
13 768.91552734375
14 263.1749572753906
15 204.44602966308594
16 531.7841186523438
17 761.6078491210938
18 613.955810546875
19 286.71014404296875
20 140.6719207763672
21 274.49505615234375
22 454.2431640625
23 435.11236572265625
24 247.64364624023438
25 113.97278594970703
26 159.90525817871094
27 281.9692687988281
28 304.7882995605469
29 203.87437438964844
30 110.16316223144531
31 125.75994110107422
32 202.42991638183594
33 223.92835998535156
34 163.0633087158203
35 101.97852325439453
36 109.99921417236328
37 156.09320068359375
38 164.7399139404297
39 122.73688507080078
40 87.068359375
41 97.5902099609375
42 125.72350311279297
43 124.1465072631836
44 95.09888458251953
45 79.35092163085938
46 92.80117797851562
47 107.71660614013672
48 99.15132

530 29.5393123626709
531 29.519512176513672
532 29.499774932861328
533 29.48008918762207
534 29.4604549407959
535 29.440876007080078
536 29.421356201171875
537 29.401880264282227
538 29.38246726989746
539 29.36311149597168
540 29.34379768371582
541 29.324546813964844
542 29.305343627929688
543 29.286195755004883
544 29.267101287841797
545 29.24806022644043
546 29.22907257080078
547 29.21013832092285
548 29.191259384155273
549 29.172426223754883
550 29.153656005859375
551 29.134929656982422
552 29.116254806518555
553 29.097639083862305
554 29.079071044921875
555 29.060558319091797
556 29.042097091674805
557 29.0236873626709
558 29.005327224731445
559 28.98702621459961
560 28.96877098083496
561 28.950572967529297
562 28.932422637939453
563 28.914323806762695
564 28.896272659301758
565 28.878280639648438
566 28.860336303710938
567 28.842443466186523
568 28.824600219726562
569 28.806806564331055
570 28.78907012939453
571 28.77137565612793
572 28.753738403320312
573 28.736148834228516
574 2

In [61]:
model[0].weight

Parameter containing:
tensor([[-9.0277e-02,  6.6201e-02, -5.2364e-02,  3.2190e+00,  1.8162e+00,
          4.4862e+00,  4.4411e-03, -9.5100e-01,  1.6331e-01, -1.0220e-02,
         -1.3492e-01,  1.4965e-02, -5.3353e-01]], requires_grad=True)

In [62]:
model[0].bias

Parameter containing:
tensor([3.0050], requires_grad=True)

In [63]:
x_temp[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [73]:
model( x[0].reshape(1,13) ) # model.forward(x[0])가능 # model( x[0].reshape(1,-1)) 가능

tensor([[29.3052]], grad_fn=<AddmmBackward0>)

In [65]:
df['medv'][0]

24.0